## Prophet Implementation

In [9]:
from prophet import Prophet
import pandas as pd
import numpy as np



In [41]:
data_set = pd.read_csv('/Users/faustin/Downloads/merged_datasets_with_events.csv',sep =',')
data_set.head()

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,...,humidity,wind_speed,rain_1h,rain_3h,snow_1h,weather_description,REF_CLOSING_DESCRIPTION,ATTENDANCE,NIGHT_SHOW_FLAG,PARADE_FLAG
0,2018-06-01,2018-06-01 19:00:00,19,2018-06-01 19:15:00.000,Roller Coaster,15,2.0,50.0,75.00,75.00,...,89,1.59,0.22,0.0,0.0,light rain,Open,46804.0,0,0
1,2018-06-01,2018-06-01 20:30:00,20,2018-06-01 20:45:00.000,Zipline,40,12.0,69.0,101.25,101.25,...,93,1.67,0.20,0.0,0.0,light rain,Open,46804.0,0,0
2,2018-06-01,2018-06-01 16:30:00,16,2018-06-01 16:45:00.000,Spinning Coaster,15,5.0,283.0,526.25,438.50,...,87,2.94,0.23,0.0,0.0,light rain,Open,46804.0,0,0
3,2018-06-01,2018-06-01 20:30:00,20,2018-06-01 20:45:00.000,Giant Wheel,30,4.0,0.0,503.75,0.00,...,93,1.67,0.20,0.0,0.0,light rain,Open,46804.0,0,0
4,2018-06-01,2018-06-01 13:45:00,13,2018-06-01 14:00:00.000,Oz Theatre,0,0.0,0.0,0.00,0.00,...,77,1.92,0.31,0.0,0.0,light rain,Fermeture Opérationnelle,46804.0,0,0


In [42]:
data_set.columns

Index(['WORK_DATE', 'DEB_TIME', 'DEB_TIME_HOUR', 'FIN_TIME',
       'ENTITY_DESCRIPTION_SHORT', 'WAIT_TIME_MAX', 'NB_UNITS',
       'GUEST_CARRIED', 'CAPACITY', 'ADJUST_CAPACITY', 'OPEN_TIME', 'UP_TIME',
       'DOWNTIME', 'NB_MAX_UNIT', 'temp', 'feels_like', 'humidity',
       'wind_speed', 'rain_1h', 'rain_3h', 'snow_1h', 'weather_description',
       'REF_CLOSING_DESCRIPTION', 'ATTENDANCE', 'NIGHT_SHOW_FLAG',
       'PARADE_FLAG'],
      dtype='object')

In [43]:
#Rename for prophet
data_set.rename(columns={'DEB_TIME' : 'ds'},inplace= True)
data_set.rename(columns={'WAIT_TIME_MAX' : 'y'},inplace= True)
data_set['ds'] = pd.to_datetime(data_set['ds'], errors='coerce')



In [44]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_set['weather_description'] = encoder.fit_transform(data_set['weather_description'])


data_set['REF_CLOSING_DESCRIPTION'] = encoder.fit_transform(data_set['REF_CLOSING_DESCRIPTION'])

In [45]:
#Creating a dataset for every attractions type 

attraction = {}

attractions = data_set['ENTITY_DESCRIPTION_SHORT'].unique().tolist()
for i in attractions:
    attraction[i] = data_set[data_set['ENTITY_DESCRIPTION_SHORT']== i].sort_values(by = 'ds', ascending = True)


In [46]:
for key in attraction:
    print(key)

Roller Coaster
Zipline
Spinning Coaster
Giant Wheel
Oz Theatre
Drop Tower
Flying Coaster
Inverted Coaster
Haunted House
Merry Go Round
Superman Ride
Crazy Dance
Dizzy Dropper
Bungee Jump
Rapids Ride
Giga Coaster
Go-Karts
Bumper Cars
Swing Ride
Free Fall
Kiddie Coaster
Spiral Slide
Circus Train
Water Ride
Himalaya Ride
Vertical Drop


In [48]:
#Implementation Prophet

 



#features
features = [
        'NB_UNITS',
       'GUEST_CARRIED', 'CAPACITY', 'ADJUST_CAPACITY', 'OPEN_TIME', 'UP_TIME',
       'DOWNTIME', 'NB_MAX_UNIT', 'temp', 'feels_like', 'humidity',
       'wind_speed', 'rain_1h', 'rain_3h', 'snow_1h', 'weather_description',
       'REF_CLOSING_DESCRIPTION', 'ATTENDANCE', 'NIGHT_SHOW_FLAG',
       'PARADE_FLAG']


model = Prophet()

for col in features:
    model.add_regressor(col)

model.fit(attraction['Spinning Coaster'][['ds','y']+features])
future = model.make_future_dataframe(periods=7*24*4, freq='15min')

print(future)

14:49:09 - cmdstanpy - INFO - Chain [1] start processing
14:49:59 - cmdstanpy - INFO - Chain [1] done processing


                       ds
0     2018-06-01 09:00:00
1     2018-06-01 09:15:00
2     2018-06-01 09:30:00
3     2018-06-01 09:45:00
4     2018-06-01 10:00:00
...                   ...
64747 2022-08-02 21:45:00
64748 2022-08-02 22:00:00
64749 2022-08-02 22:15:00
64750 2022-08-02 22:30:00
64751 2022-08-02 22:45:00

[64752 rows x 1 columns]


In [ ]:
latest_values = attraction['Spinning Coaster'].iloc[-1][features] 

for col in features:
    future[col] = latest_values[col]


forecast = model.predict(future)

print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())